In [6]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

import folium 

## Exracting the content in a wiki page that has 'List of US Cities by population' in to a text file

In [7]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)

### Finding the table that has the data that we need i.e. list of all cities with their population, Square Area, Location (coordinates)¶

In [8]:
table = soup.find_all('table')[4]

### Extracting the table from the webpage into a data frame by specifying the column names¶

In [9]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "del1", "del2", "del3", "Sq.Area", "del5", "population density(Sq Mi)", "Population density(Sq Km)", "Location"])
df1 = df.loc[[252]].shift(periods = 1 ,axis = "columns")
df1 = df1.replace(np.nan, 253, regex=True)
df.loc[[252]] = df1.loc[[252]]

### Finding the radius of each city with the help of Sq.Area, this step involves in preprocessing of the the column Sq.Area (changing its data type to float) then finding its square root¶

In [10]:
new= df["Sq.Area"].str.split("s", n=1, expand = True)
new = new[0].str.replace(u'\xa0',u'')
df["Sq.Area"] = new.str.replace(',','')
df["Sq.Area"] = df["Sq.Area"].astype(float)
df["Radius"] = np.sqrt(df["Sq.Area"])

### Drop unnecessary columns like Rank, del1, del2 ... del5, population density in Sq Mi from the data frame we extracted from the webpage¶

In [11]:
df.drop(columns = ["Rank", "del1", "del2", "del3", "del5", "Sq.Area", "population density(Sq Mi)"], inplace = True)
df.head()

,City,State,Population density(Sq Km),Location,Radius
0,New York City[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824


### Splitting the cooridnates to Latitudes and Longitudes for each city

In [12]:
df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)
Latitude = []
Longitude = []
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3]) 

k["Latitude"] = Latitude
k["Longitude"] = Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns = ["Location"], inplace = True)
df = k.copy(deep = True)
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head()

,City,State,Population density(Sq Km),Radius,Latitude,Longitude
0,New York City[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901


### Getting the per capita income state wise for USA

In [13]:
link1 = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'
page1 = requests.get(link1) 
soup1 = BeautifulSoup(page1.text)
table = soup1.find_all('table')[2]
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df_state = pd.DataFrame(res, columns=["Rank", "Country-equivalent", "State", "Per capita income", "Median", "Median2", "Population", "Households"])
df_state.head()

,Rank,Country-equivalent,State,Per capita income,Median,Median2,Population,Households
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"


### Dropping unnecessary columns like Rank, Median, Median2, Households from the Table we extracted from the webpage that has population and percapita income

In [14]:
df_state.drop(columns = ['Rank','Median', 'Median2', 'Households'], axis = 1, inplace = True)

In [15]:
map_tohood = folium.Map(location=[37.0902,-95.7129], zoom_start=3)

for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

In [16]:
CLIENT_ID = 'YM0U3GV5SDZCMDUFUNZFO3JZSZP1F4BQNNPHTJKF2DTIQCUR'
CLIENT_SECRET = 'YM2UMLJDF5IDN2TR5R1UMCYL0PRXRL4M0PP13GYMGYPELRMG'
VERSION = '20180604'
LIMIT = 20

#### Function that extracts necessary columns into a data frame from the json files that we get when we search using four square API out of the values that we have in the data frame that we got from the web page

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
df_venues = getNearbyVenues(names = df['City'], latitudes = df['Latitude'],longitudes = df['Longitude'], radius = df['Radius'])
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York City[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
1,New York City[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.668270,-73.963974,Botanical Garden
2,New York City[d],40.6635,-73.9387,Brooklyn Museum,40.671004,-73.963437,Art Museum
3,New York City[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater
4,New York City[d],40.6635,-73.9387,Prospect Park,40.661938,-73.969617,Park


### Assigning Weights to diffrent Features of each area

In [22]:
k = df_venues.copy(deep = True)
weights_dict={'Movie Theater':3,'Beach':3,'Concert Hall':2.5,'Playground':3,'Coffee Shop':3.5,'Food Court':4,'Nightclub':4,'Toy / Game Store':4.5,'Theme Park Ride / Attraction':4,'Pub':4}
data = df_venues['Venue Category']


weights = []
for i in data:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
df_venues['weights'] = weights;
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,New York City[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0.0
1,New York City[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.668270,-73.963974,Botanical Garden,0.0
2,New York City[d],40.6635,-73.9387,Brooklyn Museum,40.671004,-73.963437,Art Museum,0.0
3,New York City[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0.0
4,New York City[d],40.6635,-73.9387,Prospect Park,40.661938,-73.969617,Park,0.0



### Dropping the rows that we are not giving any weight

In [23]:
df_venues.drop(df_venues[df_venues.weights < 1.0].index, inplace=True)
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
22,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.027115,-118.387637,Coffee Shop,3.5
33,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,33.980027,-118.408020,Coffee Shop,3.5
59,Chicago,41.8376,-87.6818,Sawada Coffee,41.883730,-87.648726,Coffee Shop,3.5
73,Houston[3],29.7866,-95.3909,White Oak Music Hall,29.785994,-95.367173,Concert Hall,2.5
76,Houston[3],29.7866,-95.3909,Raven Tower,29.785364,-95.368037,Concert Hall,2.5


### Copying only the relevants columns like city and weights to group all the venues by city and calculating the means for each city

In [24]:
citywise_venues_weights = df_venues[['City','weights']].copy()
citywise_venues_weights_means = citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means = citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means.head()

,City,weights
0,Abilene,3.5
1,Alexandria[m],3.5
2,Allen,2.5
3,Allentown,4.0
4,Amarillo,3.5


### Merging the table for which we calculated the means of weights city wise to the actual table that we got from the wiki page.

In [27]:
city_selection = pd.merge(df, citywise_venues_weights_means, on='City')
city_selection = city_selection[['City','Population density(Sq Km)','weights']].copy()
city_selection.head()

,City,Population density(Sq Km),weights
0,Los Angeles,"3,276/km2",3.5
1,Chicago,"4,600/km2",3.5
2,Houston[3],"1,395/km2",2.5
3,Phoenix,"1,200/km2",3.5
4,Philadelphia[e],"4,511/km2",3.5


### Preprocessing the population density in Km2 column as we have to normalize these values

In [30]:
k = city_selection.copy(deep = True)
k['Population density(Sq Km)'] = k['Population density(Sq Km)'].str.split("/", n = 0, expand = True)
k['Population density(Sq Km)'] = k['Population density(Sq Km)'].str.replace(',','')
k['Population density(Sq Km)'] = k['Population density(Sq Km)'].astype(float)
city_selection = k.copy(deep = True)
city_selection.head()

,City,Population density(Sq Km),weights
0,Los Angeles,3276.0,3.5
1,Chicago,4600.0,3.5
2,Houston[3],1395.0,2.5
3,Phoenix,1200.0,3.5
4,Philadelphia[e],4511.0,3.5


### Normalizing the data frame

In [32]:
from sklearn import preprocessing
column_names_to_normalize = ['Population density(Sq Km)', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head()

,City,Population density(Sq Km),weights
0,Los Angeles,0.470174,0.5
1,Chicago,0.664224,0.5
2,Houston[3],0.194489,0.0
3,Phoenix,0.165909,0.5
4,Philadelphia[e],0.651180,0.5


### Calculating the sum of normalized columns to determine the city that has maximum sum and conclude that one locality in that city would be the best fit

In [33]:
city_selection['sum'] = city_selection['Population density(Sq Km)'] + city_selection['weights']
row_num = city_selection['sum'].argmax()
city_name = city_selection['City'].iloc[row_num]
city_name

'Jersey City'

### Finding the state in which that city belongs

In [35]:
row = df.loc[df['City']== city_name].index[0]
state_name = df['State'].iloc[row]
state_name

'New Jersey'

### Anthony was thinking that the state in which he will setup an arcade should've a percapital income of minimum of 50000 USD, let us check if that is the case with this

In [36]:
p_row = df_state.loc[df_state['State'] == state_name].index[0]
per_capital_income = df_state['Per capita income'].iloc[p_row]
print("Per capita income of New Jercy is :", per_capital_income)

Per capita income of New Jercy is : $50,349


## As we have concluded that we can choose one location in Jersey City, we then check for the percapita income of the state. Since it is more than 50000 USD, it is considered as one of the best places to procced for setting up a Gaming arcade

### Let us now check which location in Jersey City would be the best to start a Gaming arcade

In [37]:
lat_newJercy = df['Latitude'].iloc[row]
long_newJercy = df['Longitude'].iloc[row]
print(lat_newJercy, long_newJercy)

40.7114 -74.0648


### Getting all the venues of New Jersey using four square API within our considered radius and Limits

In [38]:
def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


new_jersey_venues = getNearbyVenues1(name = 'Jersey City', latitudes = lat_newJercy ,longitudes = long_newJercy, radius = 2500)

In [39]:
venues_in_newjersey = new_jersey_venues.copy(deep = True)
venues_in_newjersey.shape

(100, 7)

### Since we have got 100 categories, we are now giving weights for each category for better results

In [40]:
k = new_jersey_venues.copy(deep = True)
new_weightage_dict= {'Coffee Shop' : 3, 
'Caribbean Restaurant':3,
'Distillery':2,
'Fish & Chips Shop':3,
'Science Museum':3,
'Latin American Restaurant':4,
'Restaurant':5,
'State / Provincial Park':1,
'Diner':1,
'Supermarket':1,
'Bar':1,
'Jazz Club':1,
'Golf Course':3,
'Park':2,
'Cajun / Creole Restaurant':2,
'Bakery':2,
'Go Kart Track':3,
'Taco Place':3,
'Hot Dog Joint':2,
'Food Truck':3,
'Beer Garden':3,
'Boutique':4,
'Café':5,
'Bagel Shop':1,
'Record Shop':1,
'Bakery':1,
'Pizza Place':1,
'Ramen Restaurant':1,
'Wine Bar':3,
'Middle Eastern Restaurant':2,
'French Restaurant':2,
'Theater':2,
'Lounge':3,
'Wine Shop':3,
'Cocktail Bar':2,
'New American Restaurant':3,
'Residential Building (Apartment / Condo)':3,
'Pool':4,
'Burger Joint':5,
'Cheese Shop':1,
'Coffee Shop':1,
'Bagel Shop':1,
'Vietnamese Restaurant':1,
'Portuguese Restaurant':1,
'Ice Cream Shop':3,
'Italian Restaurant':2,
'Gym':2,
'Farmers Market':2,
'Bar':3,
'Pizza Place':3,
'Bakery':2,
'Bookstore':3,
'Bar':3,
'Farmers Market':4,
'Asian Restaurant':5,
'Tea Room':1,
'Donut Shop':1,
'Historic Site':1,
'Gym / Fitness Center':1,
'Café':1,
'Mexican Restaurant':3,
'Plaza':2,
'Gay Bar':2,
'Bar':3,
'College Administrative Building':3,
'Mexican Restaurant':2,
'Bakery':3,
'American Restaurant':3,
'American Restaurant':4,
'American Restaurant':5,
'Café':1,
'New American Restaurant':1,
'Chocolate Shop':1,
'Gym':1,
'Grocery Store':1,
'Middle Eastern Restaurant':3,
'American Restaurant':2,
'Frozen Yogurt Shop':2,
'Japanese Restaurant':2,
'Bar':3,
'Liquor Store':3,
'Ice Cream Shop':2,
'Fish Market':3,
'Indie Movie Theater':3,
'Grocery Store':4,
'Modern European Restaurant':5,
'American Restaurant':1,
'Poke Place':1,
'Ramen Restaurant':1,
'Diner':1,
'Brewery':1,
'Burger Joint':3,
'Burger Joint':2,
'Café':2,
'Fried Chicken Joint':2,
'Beer Garden':3,
'Gym / Fitness Center':3,
'Vietnamese Restaurant':2,
'Italian Restaurant':3,
'Pet Store':3}

In [42]:
venues_map = folium.Map(location=[lat_newJercy, long_newJercy], zoom_start=15) 

folium.CircleMarker(
    [lat_newJercy, long_newJercy],
    radius=10,
    popup='Jersey city',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

In [44]:
for lat, lng, label in zip(venues_in_newjersey['Venue Latitude'], venues_in_newjersey['Venue Longitude'], venues_in_newjersey['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map

### Assigning weights to each category, same as we gave for each city

In [45]:
allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,Jersey City,40.5607,-74.2927,Harry’s Daughter,40.710904,-74.062071,Caribbean Restaurant,3
1,Jersey City,40.5607,-74.2927,The Grind Shop,40.711670,-74.062872,Coffee Shop,1
2,Jersey City,40.5607,-74.2927,Corgi Spirits at The Jersey City Distillery,40.708304,-74.064803,Distillery,2
3,Jersey City,40.5607,-74.2927,Mordis Sandwich Shop,40.710412,-74.061095,Sandwich Place,0
4,Jersey City,40.5607,-74.2927,Hooked JC,40.714709,-74.067009,Fish & Chips Shop,3


### Dropping unnecessary columns

In [47]:
newframe = k[['City','Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe.head()

,Venue Latitude,Venue Longitude,weights
Venue Category,,,
Afghan Restaurant,40.709155,-74.073795,0
American Restaurant,40.718157,-74.046589,1
Australian Restaurant,40.717187,-74.044216,0
Bagel Shop,40.722990,-74.058068,1
Bakery,40.725514,-74.051158,3


### Using K Means algorithm to cluster the venues and calculating the weights for each cluster to decide which cluster would be the best area to install a gaming arcade¶

In [50]:
# Cluster them using K means algorithm 
from scipy import stats
import matplotlib.pyplot as plt

#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])

#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels

#Add the column into our list
clmns.extend(['clusters'])

#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Venue Category,weights,Venue Latitude,Venue Longitude,clusters
0,Afghan Restaurant,0,40.709155,-74.073795,2
1,American Restaurant,1,40.718157,-74.046589,0
2,Australian Restaurant,0,40.717187,-74.044216,0
3,Bagel Shop,1,40.722990,-74.058068,0
4,Bakery,3,40.725514,-74.051158,1


### New group by clusters and add weights of each cluster 

In [51]:
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,0.583333,40.718331,-74.051640
1,2.727273,40.720715,-74.047847
2,2.454545,40.708745,-74.068460


In [52]:
lat1 = 40.720102
long1 = -74.048121

### As we have a location with the maximum weight, let us plot the same in the graph with a circle of radius 50M such that Vasanth can install his arcade with in that premices

In [54]:
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Jersey city


# add prefered location in the City as a green circle mark
folium.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='Gaming arcade can be installed within this circle',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map

### So, we finally got a better place in the Jersey city

# This place is between the Groove Street and the Grand Street, hence it would also have the best footfall and potential customers as well.

## Further Enhancements and drawbacks in continuing this approach :

### This project can be enhanced by considering many more attributes to define the weights and do the analysis and also by extending the LIMIT and Radius of the search that we are giving to extract the number of venues. As we have an API limit in the free trail of four square API we had to limit our search within a small Radius.

### Also, further enhancements could be, we have hardcorded the radius to 2500 around the Jercey city but the city radius might be more or less than that (actually more). Hence based on the weights we gave our cluster might go to other city if the radius is more than what it actually is.